In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [4]:
test_data = pd.read_pickle("test_data_Lenskit2")
all_recs_joined = pd.read_pickle("all_recs_joined_Lenskit2")

In [5]:
testdata = all_recs_joined.loc[(all_recs_joined["user"] == 1) | (all_recs_joined["user"] == 1)]
testdata = all_recs_joined.loc[(all_recs_joined["user"] == 1) & (all_recs_joined["Algorithm"] == "ALS")]
testdata = testdata.iloc[0:100,:] 
#testdata.loc[testdata["Action"]==1].shape 


In [6]:
def evaluate_genre(genre): 
    
    providers = list(all_recs_joined.iloc[:,8:])
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    #rla.add_metric("rKL")
    #rla.add_metric("rRD")
    #rla.add_metric("APCR")
    rla.add_metric("nd_APCR")
    #rla.add_metric("equal_ex")
    #rla.add_metric("ndcg")
    results= rla.compute(testdata, test_data, genre, providers)
    #results_horror.groupby('Algorithm').mean()
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [7]:
testproviders = ["Action"]
#testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)    

Action
calculate normalizer :  100 + 18 + rND
0.6050285048051399
done Action


C:\Users\josse\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [8]:
output = pd.concat(allResults, keys=testproviders)
output

,,rND,nd_APCR
,Algorithm,,
Action,ALS,0.207064,0.771326


In [9]:
#topnFair.getNormalizer(,7, "rND")

In [10]:
(((4/10)-(18/100))/math.log(11,2))+(((6/20)-(18/100))/math.log(21,2))

0.09091469163356969

In [11]:
(((4/10)-(18/100))/math.log(11,2))+(((6/20)-(18/100))/math.log(21,2))+((7/30)-(18/100))/math.log(31,2)+((9/40)-(18/100))/math.log(41,2)+((11/50)-(18/100))/math.log(51,2)+((12/60)-(18/100))/math.log(61,2)+((14/70)-(18/100))/math.log(71,2)+((15/80)-(18/100))/math.log(81,2)+((16/90)-(18/100))/math.log(91,2)+((18/100)-(18/100))/math.log(101,2)


0.12459694278133669